# 4. Batch Inference Pipeline

## 4.1. Environment Setup
Detect if running in Google Colab or local environment, handle repository cloning, dependency installation, numpy compatibility fixes, and set up Python path.

In [87]:
import sys
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    try:
        if "google.colab" in str(get_ipython()):
            return True
    except:
        pass
    return False

def clone_repository() -> None:
    repo_dir = Path("pm25-forecast-openmeteo-aqicn")
    if repo_dir.exists():
        print(f"Repository already exists at {repo_dir.absolute()}")
        %cd pm25-forecast-openmeteo-aqicn
    else:
        print("Cloning repository...")
        !git clone https://github.com/KristinaPalmquist/pm25-forecast-openmeteo-aqicn.git
        %cd pm25-forecast-openmeteo-aqicn

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

def fix_numpy_compatibility():
    print("Fixing numpy compatibility for hopsworks/pandas...")
    try:
        # Use compatible versions that work with the installed packages
        !pip install --force-reinstall numpy==1.26.4 pandas==2.0.3
        print("Numpy and pandas fixed. Please restart runtime and run again.")
    except Exception as e:
        print(f"Fix attempt failed: {e}")
        print("Please manually restart runtime and try again.")

if is_google_colab():
    try:
        import numpy
        numpy.array([1, 2, 3])
        import pandas as pd
        print("Basic packages working correctly")

        clone_repository()
        install_dependencies()

        import hopsworks
        print("All packages working correctly")

        root_dir = str(Path().absolute())
        print("Google Colab environment")
        
    except (ValueError, ImportError) as e:
        if "numpy.dtype size changed" in str(e) or "numpy.strings" in str(e) or "numpy" in str(e).lower():
            fix_numpy_compatibility()
            raise SystemExit("Please restart runtime (Runtime > Restart runtime) and run the notebook again.")
        else:
            raise

else:
    root_dir = Path().absolute()
    if root_dir.parts[-1:] == ("src",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("airquality",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("notebooks",):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir)
    print("Local environment")

print(f"Root dir: {root_dir}")

if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

if is_google_colab():
    from google.colab import userdata
    import hopsworks
    project = hopsworks.login(
        api_key_value=userdata.get('HOPSWORKS_API_KEY'),
        engine="python"
    )
    AQICN_API_KEY = userdata.get('AQICN_API_KEY')
    
else:
    from utils import config
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn
HopsworksSettings initialized!


## 4.2. Imports

In [88]:
# from datetime import datetime, timedelta
import datetime
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import hopsworks
import json
from utils import airquality
from scipy.spatial.distance import cdist
import matplotlib.colors as mcolors
# import matplotlib.pyplot as plt
import os

warnings.filterwarnings("ignore")

## 4.3. Hopsworks Configuration
Establish connection to Hopsworks, retrieve API keys, connect to feature store, and get air quality and weather feature groups.

In [89]:
if is_google_colab():
    fs = project.get_feature_store()
    secrets = hopsworks.get_secrets_api()
else:
    HOPSWORKS_API_KEY = getattr(settings, 'HOPSWORKS_API_KEY', None)

    if HOPSWORKS_API_KEY is not None and hasattr(HOPSWORKS_API_KEY, 'get_secret_value'):
        HOPSWORKS_API_KEY = HOPSWORKS_API_KEY.get_secret_value()

    project = hopsworks.login(engine="python", api_key_value=HOPSWORKS_API_KEY)

    fs = project.get_feature_store()

    secrets = hopsworks.get_secrets_api()
    AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value


today = datetime.datetime.today().date()
past_date = today - datetime.timedelta(days=4)

# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name="air_quality_all",
    version=1,
)

weather_fg = fs.get_feature_group(
    name="weather_all",
    version=1,
)

2025-12-09 11:14:53,158 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-12-09 11:14:53,165 INFO: Initializing external client
2025-12-09 11:14:53,165 INFO: Base URL: https://c.app.hopsworks.ai:443
Connection closed.
2025-12-09 11:14:53,165 INFO: Initializing external client
2025-12-09 11:14:53,165 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-12-09 11:14:54,504 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184


## 4.4. Sensor Location Loading 
Load sensor location metadata from Hopsworks secrets for all sensors.

In [90]:
all_secrets = secrets.get_secrets()
locations = {}
for secret in all_secrets:
    if secret.name.startswith("SENSOR_LOCATION_JSON_"):
        sensor_id = secret.name.replace("SENSOR_LOCATION_JSON_", "")
        location_str = secrets.get_secret(secret.name).value
        if location_str:
            locations[sensor_id] = json.loads(location_str)
print(f"Retrieved locations for {len(locations)} sensors from Hopsworks Secrets Manager.")

Retrieved locations for 105 sensors from Hopsworks Secrets Manager.


## 4.5. Weather Data Loading
Fetch recent weather data from feature store and convert date formats

In [ ]:
try:
    batch_weather = weather_fg.filter(weather_fg.date >= past_date).read()
except Exception:
    batch_weather = weather_fg.read()
    batch_weather = batch_weather[batch_weather["date"] >= past_date]
batch_weather["date"] = pd.to_datetime(batch_weather["date"]).dt.tz_localize(None)
print(f"Retrieved {len(batch_weather)} weather records from Hopsworks Feature Store.")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.97s) 
Retrieved 945 weather records from Hopsworks Feature Store.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.97s) 
Retrieved 945 weather records from Hopsworks Feature Store.


## 4.6. Air Quality Data Loading
Fetch recent air quality with error handling for missing data.

In [92]:
try:
    batch_airquality = air_quality_fg.filter(air_quality_fg.date >= past_date).read()
    batch_airquality["date"] = pd.to_datetime(batch_airquality["date"]).dt.tz_localize(None)
except Exception:
    batch_airquality = pd.DataFrame()
print(f"Retrieved {len(batch_airquality)} air quality records from Hopsworks Feature Store.")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.84s) 
Retrieved 105 air quality records from Hopsworks Feature Store.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.84s) 
Retrieved 105 air quality records from Hopsworks Feature Store.


## 4.7. Model Retrieval
Download trained XGBoost models from Hopsworks model registry for each sensor and extract feature names.

In [ ]:
mr = project.get_model_registry()

MODEL_NAME_TEMPLATE = "air_quality_xgboost_model_{sensor_id}"

# model, model_dir, features
retrieved_models = {}

for sensor_id in locations.keys():
    model_name = MODEL_NAME_TEMPLATE.format(sensor_id=sensor_id)
    retrieved_model = None

    available_models = mr.get_models(name=model_name)
    if available_models:
        retrieved_model = max(available_models, key=lambda model: model.version)
    # 105 models found

    if retrieved_model is None:
        print(f"No model found for sensor {sensor_id}, skipping...")
        continue
    
    saved_model_dir = retrieved_model.download()
    # xgb_model = XGBRegressor()
    # xgb_model.load_model(saved_model_dir + "/model.json")
    # booster = xgb_model.get_booster()
    
    import xgboost as xgb
    booster = xgb.Booster()
    booster.load_model(saved_model_dir + "/model.json")
    xgb_model = XGBRegressor()
    xgb_model._Booster = booster

    retrieved_models[sensor_id] = retrieved_model, xgb_model, booster.feature_names

Downloading: 0.000%|          | 0/558662 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/120639 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/41984 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/38974 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/24122 elapsed<00:00 remaining<?

...

Downloading: 0.000%|          | 0/561530 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/59334 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/41611 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/41982 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/20706 elapsed<00:00 remaining<?

In [94]:
print(f"Retrieved models length {len(retrieved_models)}.")

Retrieved models length 105.


## 4.8. Batch Prediction Loop
Merge weather and air quality data, iteratively predict PM2.5 values for forecast days, update engineered features after each prediction, and store results

In [95]:
PREDICTION_CAP_MAX = 150.0  # Maximum reasonable PM2.5 value
PREDICTION_CAP_MIN = 0.0    # Minimum reasonable PM2.5 value
extreme_predictions = []  # Track extreme predictions for investigation

# Merge historical data with weather data
batch_data = pd.merge(batch_weather, batch_airquality, on=["date", "sensor_id"], how="left")
batch_data = batch_data.sort_values(["sensor_id", "date"])

feature_cols = [
    "pm25_rolling_3d",
    "pm25_lag_1d",
    "pm25_lag_2d",
    "pm25_lag_3d",
    "pm25_nearby_avg",
]

# # Apply initial feature engineering to all historical data (needed for first predictions)
# batch_data = airquality.add_rolling_window_feature(
#     batch_data, window_days=3, column="pm25", new_column="pm25_rolling_3d"
# )
# batch_data = airquality.add_lagged_features(batch_data, column="pm25", lags=[1, 2, 3])
# batch_data = airquality.add_nearby_sensor_feature(
#     batch_data,
#     locations,
#     column="pm25",
#     n_closest=3,
#     new_column="pm25_nearby_avg",
# )

# # Create a composite PM2.5 column that uses actual values when available, predictions otherwise
# batch_data["pm25_composite"] = batch_data["pm25"].fillna(0)  # Initialize with actual values
batch_data["predicted_pm25"] = np.nan
batch_data["days_before_forecast_day"] = np.nan
for col in feature_cols:
    batch_data[f"predicted_{col}"] = np.nan

forecast_days = (
    batch_data.loc[batch_data["pm25"].isna() & (batch_data["date"] >= today.strftime("%Y-%m-%d")), "date"]
    .dropna()
    .sort_values()
    .unique()
)

# print(f"📅 Processing {len(forecast_days)} forecast days: {[day.strftime('%Y-%m-%d') for day in forecast_days]}")

# # Track sensors processed to prevent duplicates and count progress
# sensors_processed = set()
# warning_count = 0
# MAX_WARNINGS = 3  # Reduced to minimize output noise
# predictions_made = 0

# for day_idx, target_day in enumerate(forecast_days):
#     print(f"🔄 Day {day_idx + 1}/{len(forecast_days)}: {target_day.strftime('%Y-%m-%d')}", end="")
#     daily_predictions = 0
    
#     # context with all sensors up to current day - use composite column for feature engineering
#     temp_batch_data = batch_data.copy()
    
#     # Update composite column with predictions made so far
#     prediction_mask = temp_batch_data["predicted_pm25"].notna()
#     temp_batch_data.loc[prediction_mask, "pm25_composite"] = temp_batch_data.loc[prediction_mask, "predicted_pm25"]
    
#     # Recompute features using the composite column (historical + predictions made so far)
#     temp_batch_data = airquality.add_rolling_window_feature(
#         temp_batch_data, window_days=3, column="pm25_composite", new_column="pm25_rolling_3d"
#     )
#     temp_batch_data = airquality.add_lagged_features(temp_batch_data, column="pm25_composite", lags=[1, 2, 3])
#     temp_batch_data = airquality.add_nearby_sensor_feature(
#         temp_batch_data,
#         locations,
#         column="pm25_composite",
#         n_closest=3,
#         new_column="pm25_nearby_avg",
#     )
    
#     # Get rows for this target day that need predictions
#     day_rows = temp_batch_data[(temp_batch_data["date"] == target_day) & temp_batch_data["pm25"].isna()]

#     for _, row in day_rows.iterrows():
#         sensor_id = row["sensor_id"]
        
#         # Create unique key for this sensor-day combination
#         sensor_day_key = f"{sensor_id}_{target_day.strftime('%Y-%m-%d')}"
        
#         # Skip if already processed to prevent duplicates
#         if sensor_day_key in sensors_processed:
#             continue
            
#         # Mark as processed immediately
#         sensors_processed.add(sensor_day_key)

#         # Skip sensors that don't have models
#         if sensor_id not in retrieved_models:
#             continue

#         _, xgb_model, model_features = retrieved_models[sensor_id]
#         features = (row.reindex(model_features).to_frame().T.apply(pd.to_numeric, errors="coerce"))
        
#         # Enhanced feature analysis
#         pm25_feature_cols = [col for col in model_features if 'pm25' in col]
#         valid_pm25_features = 0
        
#         if len(pm25_feature_cols) > 0:
#             valid_pm25_features = (~features[pm25_feature_cols].isna()).sum().iloc[0]
            
#             # Skip only if we have NO valid PM2.5 features at all
#             if valid_pm25_features == 0:
#                 if warning_count < MAX_WARNINGS:
#                     print(f"\n⚠️  Skipping sensor {sensor_id} on {target_day.strftime('%Y-%m-%d')}: No PM2.5 features")
#                     warning_count += 1
#                 continue
        
#         # Make prediction for this sensor/day
#         y_hat_raw = xgb_model.predict(features)[0]
        
#         # Collect diagnostic info for extreme predictions
#         if y_hat_raw > PREDICTION_CAP_MAX or y_hat_raw < PREDICTION_CAP_MIN:
#             extreme_predictions.append({
#                 'sensor_id': sensor_id,
#                 'date': target_day,
#                 'raw_prediction': y_hat_raw,
#                 'features': features.iloc[0].to_dict()
#             })
        
#         # Cap predictions to reasonable range
#         y_hat = np.clip(y_hat_raw, PREDICTION_CAP_MIN, PREDICTION_CAP_MAX)
        
#         # Only show capping warnings for first few instances
#         if y_hat != y_hat_raw and warning_count < MAX_WARNINGS:
#             print(f"\n⚠️  Capped prediction: {y_hat_raw:.1f} → {y_hat:.1f}")
#             warning_count += 1

#         idx = batch_data.index[(batch_data["sensor_id"] == sensor_id) & (batch_data["date"] == target_day)][0]
#         batch_data.at[idx, "predicted_pm25"] = y_hat
#         batch_data.at[idx, "days_before_forecast_day"] = (target_day.date() - today).days
        
#         predictions_made += 1
#         daily_predictions += 1
    
#     # Concise daily summary
#     print(f" → {daily_predictions} predictions ✅")

# # Final processing summary
# print(f"\n🎯 Prediction Summary:")
# print(f"  📈 Total predictions: {predictions_made}")
# print(f"  📅 Days processed: {len(forecast_days)}")
# print(f"  🔧 Unique sensor-day combinations: {len(sensors_processed)}")

# # Extract predictions while preserving predicted_pm25 values
# predictions = batch_data.loc[
#     batch_data["predicted_pm25"].notna(),
#     ["date", "sensor_id", "predicted_pm25", "days_before_forecast_day"],
# ].reset_index(drop=True)

# # Add the engineered features using the composite approach for final output
# final_batch_data = batch_data.copy()
# prediction_mask = final_batch_data["predicted_pm25"].notna()
# final_batch_data.loc[prediction_mask, "pm25_composite"] = final_batch_data.loc[prediction_mask, "predicted_pm25"]

# # Recompute all features one final time for the predictions output
# final_batch_data = airquality.add_rolling_window_feature(
#     final_batch_data, window_days=3, column="pm25_composite", new_column="pm25_rolling_3d"
# )
# final_batch_data = airquality.add_lagged_features(final_batch_data, column="pm25_composite", lags=[1, 2, 3])
# final_batch_data = airquality.add_nearby_sensor_feature(
#     final_batch_data,
#     locations,
#     column="pm25_composite",
#     n_closest=3,
#     new_column="pm25_nearby_avg",
# )

# # Add the computed features to predictions
# for _, pred_row in predictions.iterrows():
#     sensor_id = pred_row['sensor_id']
#     date = pred_row['date']
    
#     final_row = final_batch_data[(final_batch_data['sensor_id'] == sensor_id) & (final_batch_data['date'] == date)]
#     if not final_row.empty:
#         for col in feature_cols:
#             predictions.loc[predictions.index[_], f"predicted_{col}"] = final_row[col].iloc[0]

# print(f"  💾 Generated predictions DataFrame with {len(predictions)} rows")

# if len(predictions) > 0:
#     print(f"  ✅ Generated {len(predictions)} predictions from {predictions['date'].min().strftime('%Y-%m-%d')} to {predictions['date'].max().strftime('%Y-%m-%d')}")
#     print(f"  📊 Sensors with predictions: {predictions['sensor_id'].nunique()}")
# else:
#     print(f"  ⚠️  No predictions generated - check sensor data and models")

In [98]:
for day in forecast_days:
    print(type(day.date()))
print(type(today))

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>


In [103]:
for target_day in forecast_days:
    # context with all sensors up to current day
    window = batch_data.loc[batch_data["date"] <= target_day].copy()
    day_rows = window[(window["date"] == target_day) & window["pm25"].isna()]

    for _, row in day_rows.iterrows():
        sensor_id = row["sensor_id"]

        _, xgb_model, model_features = retrieved_models[sensor_id]
        features = (row.reindex(model_features).to_frame().T.apply(pd.to_numeric, errors="coerce"))
        y_hat = xgb_model.predict(features)[0]

        idx = batch_data.index[(batch_data["sensor_id"] == sensor_id) & (batch_data["date"] == target_day)][0]
        batch_data.at[idx, "pm25"] = y_hat
        batch_data.at[idx, "predicted_pm25"] = y_hat
        batch_data.at[idx, "days_before_forecast_day"] = (target_day.date() - today).days + 1

    # recompute features for all sensors now that this days values exist
    temp_df = batch_data.loc[batch_data["date"] <= target_day].copy()
    temp_df = airquality.add_rolling_window_feature(
        temp_df, window_days=3, column="pm25", new_column="pm25_rolling_3d"
    )
    temp_df = airquality.add_lagged_features(temp_df, column="pm25", lags=[1, 2, 3])
    temp_df = airquality.add_nearby_sensor_feature(
        temp_df,
        locations,
        column="pm25",
        n_closest=3,
        new_column="pm25_nearby_avg",
    )

    current_rows = temp_df[temp_df["date"] == target_day]
    for _, row in current_rows.iterrows():
        sensor_id = row["sensor_id"]
        mask = (batch_data["sensor_id"] == sensor_id) & (batch_data["date"] == target_day)
        if mask.any():
            for col in feature_cols:
                batch_data.loc[mask, f"predicted_{col}"] = row[col]

predictions = batch_data.loc[
    batch_data["predicted_pm25"].notna(),
    ["date", "sensor_id", "predicted_pm25", "days_before_forecast_day"]
    + [f"predicted_{col}" for col in feature_cols],
].reset_index(drop=True)
batch_data.loc[batch_data["date"].dt.date > today, "pm25"] = np.nan

## 4.9. Save Predictions
Export prediction results to CSV file in models directory.

In [104]:
batch_data.to_csv(f"{root_dir}/models/predictions.csv", columns=batch_data.columns, index=False)

In [105]:
# # Add location data directly from locations dictionary
# for sensor_id, location in locations.items():
#     sensor_mask = predictions['sensor_id'] == sensor_id
#     predictions.loc[sensor_mask, 'latitude'] = location['latitude']
#     predictions.loc[sensor_mask, 'longitude'] = location['longitude']
#     predictions.loc[sensor_mask, 'city'] = location['city']
#     predictions.loc[sensor_mask, 'street'] = location['street']

# predictions.to_csv(f"{root_dir}/models/predictions.csv", index=False)
# print(f"✅ Saved {len(predictions)} predictions with location data to CSV")

## 4.10. Generate Forecast Plots
Create forecast visualization plots for each sensor and upload them to Hopsworks dataset storage.

In [106]:
forecast_paths = []

for sensor_id, location in locations.items():
    sensor_forecast = predictions[predictions["sensor_id"] == sensor_id].copy()

    city, street = location["city"], location["street"]
    forecast_path = f"{root_dir}/models/{sensor_id}/images/forecast.png"
    Path(forecast_path).parent.mkdir(parents=True, exist_ok=True)

    plt = airquality.plot_air_quality_forecast(
        location["city"],
        location["street"],
        sensor_forecast,
        forecast_path,
        hindcast=False,
    )
    plt.close()
    forecast_paths.append((sensor_id, forecast_path))

dataset_api = project.get_dataset_api()
today_short = today.strftime("%Y-%m-%d")
if not dataset_api.exists("Resources/airquality"):
    dataset_api.mkdir("Resources/airquality")

for sensor_id, forecast_path in forecast_paths:
    dataset_api.upload(
        forecast_path,
        f"Resources/airquality/{sensor_id}_{today_short}_forecast.png",
        overwrite=True,
    )
print(f"Forecast plots available in Hopsworks under {project.get_url()}/settings/fb/path/Resources/airquality")

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/121810/images/forecast.png: 0.0…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/192520/images/forecast.png: 0.0…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/196735/images/forecast.png: 0.0…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/208483/images/forecast.png: 0.0…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/415030/images/forecast.png: 0.0…

...

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/77488/images/forecast.png: 0.00…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/79999/images/forecast.png: 0.00…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/81505/images/forecast.png: 0.00…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/92683/images/forecast.png: 0.00…

Forecast plots available in Hopsworks under https://c.app.hopsworks.ai:443/p/1279184/settings/fb/path/Resources/airquality


## 4.11. Insert Monitoring Data
Save predictions to monitoring feature group in Hopsworks for tracking.

In [107]:
monitor_fg = fs.get_or_create_feature_group(
    name="aq_predictions",
    description="Air Quality prediction monitoring",
    version=1,
    primary_key=["sensor_id", "date", "days_before_forecast_day"],
    event_time="date",
)

# # Remove location columns that are not part of the monitoring schema
# monitoring_predictions = predictions.drop(columns=['latitude', 'longitude', 'city', 'street'], errors='ignore')
# monitor_fg.insert(monitoring_predictions, wait=True)
monitor_fg.insert(predictions, wait=True)


Uploading Dataframe: 100.00% |██████████| Rows 630/630 | Elapsed Time: 00:01 | Remaining Time: 00:00



Launching job: aq_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279184/jobs/named/aq_predictions_1_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279184/jobs/named/aq_predictions_1_offline_fg_materialization/executions
2025-12-09 11:38:29,526 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-12-09 11:38:29,526 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-12-09 11:38:35,931 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-12-09 11:38:35,931 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-12-09 11:40:15,343 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: SUCCEEDED
2025-12-09 11:40:15,343 INFO: Waiting for executi

(Job('aq_predictions_1_offline_fg_materialization', 'SPARK'), None)

## 4.12. Hindcast Analysis
Compare predicted with forecasted values (1-day prior forecast)

In [108]:
monitoring_df = monitor_fg.filter(monitor_fg.days_before_forecast_day == 1).read()
monitoring_df["date"] = pd.to_datetime(monitoring_df["date"]).dt.tz_localize(None)

air_quality_df = air_quality_fg.read()[["date", "sensor_id", "pm25"]]
air_quality_df["date"] = pd.to_datetime(air_quality_df["date"]).dt.tz_localize(None)

for sensor_id, location in locations.items():
    try:
        sensor_preds = monitoring_df[monitoring_df["sensor_id"] == sensor_id][["date", "predicted_pm25"]]
        merged = sensor_preds.merge(
            air_quality_df[air_quality_df["sensor_id"] == sensor_id][["date", "pm25"]],
            on="date",
            how="inner",
        ).sort_values("date")

        city, street = location["city"], location["street"]
        hindcast_path = f"{root_dir}/models/{sensor_id}/images/hindcast_prediction.png"
        Path(hindcast_path).parent.mkdir(parents=True, exist_ok=True)

        plt = airquality.plot_air_quality_forecast(
            city,
            street,
            merged if not merged.empty else sensor_preds.assign(pm25=np.nan),
            hindcast_path,
            hindcast=True,
        )
        plt.close()

        dataset_api.upload(
            hindcast_path,
            f"Resources/airquality/{sensor_id}_{today:%Y-%m-%d}_hindcast.png",
            overwrite=True,
        )

        # # Check if file was created successfully before uploading
        # if Path(hindcast_path).exists() and Path(hindcast_path).stat().st_size > 0:
        #     try:
        #         dataset_api.upload(
        #             hindcast_path,
        #             f"Resources/airquality/{sensor_id}_{today:%Y-%m-%d}_hindcast.png",
        #             overwrite=True,
        #         )
        #         print(f"✅ Uploaded hindcast for sensor {sensor_id}")
        #     except Exception as upload_error:
        #         print(f"⚠️  Failed to upload hindcast for sensor {sensor_id}: {upload_error}")
        # else:
        #     print(f"⚠️  Hindcast file not created or empty for sensor {sensor_id}")
            
    except Exception as e:
        print(f"⚠️  Error processing hindcast for sensor {sensor_id}: {e}")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.13s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.13s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.70s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.70s) 


Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/121810/images/hindcast_predicti…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/192520/images/hindcast_predicti…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/196735/images/hindcast_predicti…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/208483/images/hindcast_predicti…

...

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/70564/images/hindcast_predictio…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/77488/images/hindcast_predictio…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/79999/images/hindcast_predictio…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/81505/images/hindcast_predictio…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/92683/images/hindcast_predictio…

## 4.13 IDW Heatmap
IDW - Inverse Distance Weighting

### 4.13.1 IDW interpolation function

In [109]:
def idw_interpolation(points, values, grid_points, lon_mesh, power=2):
    # compute distances between grid points and known data points 
    distances = cdist(grid_points, points)
    # replace 0 with a small value to avoid division by zero
    distances = np.where(distances == 0, 1e-10, distances)
    # compute weights based on inverse distance
    weights = 1.0 / (distances ** power)
    # sum of weights for normalization
    weights_sum = np.sum(weights, axis=1)
    # compute interpolated values - weighted average of known values for each grid point
    interpolated = np.sum(weights * values, axis=1) / weights_sum
    # reshape to the match grid shape
    return interpolated.reshape(lon_mesh.shape)

In [110]:
def plot_pm25_idw_heatmap(
    predictions: pd.DataFrame,
    locations: dict,
    forecast_date: datetime.datetime,
    path: str,
    grid_bounds=(-7.602536,50.862218,36.738284,69.923179),
    grid_resolution=800,
    power=2,
):

    df_day = predictions[predictions["date"] == forecast_date].copy()

    sensor_coords = np.array([[locations[sid]["longitude"], locations[sid]["latitude"]]
                              for sid in df_day["sensor_id"].unique() if sid in locations])

    pm25_column = "predicted_pm25"
    if df_day["predicted_pm25"].isna().any():
        pm25_column = "pm25"

    pm25_values = np.array([df_day[df_day["sensor_id"] == sid][pm25_column].iloc[0]
                            for sid in df_day["sensor_id"].unique() if sid in locations])
    
    # Cap extreme values to prevent unrealistic interpolation
    pm25_values = np.clip(pm25_values, 0, 150)

    min_lon, min_lat, max_lon, max_lat = grid_bounds

    lon_grid = np.linspace(min_lon, max_lon, grid_resolution)
    lat_grid = np.linspace(min_lat, max_lat, grid_resolution)
    lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)
    grid_points = np.column_stack([lon_mesh.ravel(), lat_mesh.ravel()])

    idw_result = idw_interpolation(sensor_coords, pm25_values, grid_points, lon_mesh, power=power)

    default_levels = np.array([0, 12, 35, 55, 150, 250, 500])
    category_colors = [
            "#00e400", "#7de400", "#ffff00", "#ffb000",
            "#ff7e00", "#ff4000", "#ff0000", "#d00050",
            "#8f3f97", "#7e0023"
        ]
    # category_colors = ["#00e400", "#7de400", "#ffff00", "#ffb000", "#ff7e00", "#ff4000", "#ff0000", "#c0007f", "#8f3f97", "#7e0023"]
    vmin, vmax = default_levels[0], 150
    
    clipped = np.clip(idw_result, vmin, vmax)
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(
        clipped,
        extent=(min_lon, max_lon, min_lat, max_lat),
        origin="lower",
        cmap=mcolors.LinearSegmentedColormap.from_list("aqi", category_colors, N=512),
        vmin=vmin,
        vmax=vmax,
        alpha=0.5,
    )
    ax.set_xlim(min_lon, max_lon)
    ax.set_ylim(min_lat, max_lat)
    ax.axis("off")

    fig.savefig(path, dpi=300, bbox_inches="tight", pad_inches=0, transparent=True)
    plt.close(fig)

In [111]:
interpolation_dir = f"{root_dir}/models/interpolation"
if not os.path.exists(interpolation_dir):
    os.mkdir(interpolation_dir)

today_short = today.strftime("%Y-%m-%d")

interpolation_df = batch_data[batch_data["date"] >= today_short]
for i, forecast_date in enumerate(sorted(interpolation_df["date"].unique())):
    forecast_date_short = forecast_date.strftime("%Y-%m-%d")
    output_png = f"{interpolation_dir}/forecast_interpolation_{i}d.png"
    
    plot_pm25_idw_heatmap(
        interpolation_df,
        locations,
        forecast_date,
        output_png,
    )
    dataset_api.upload(
        output_png,
        f"Resources/airquality/interpolation_{today_short}_{forecast_date_short}.png",
        overwrite=True,
    )

# # Use predictions DataFrame which contains all forecast days with PM2.5 values
# interpolation_df = predictions.copy()
# # Add any actual PM2.5 data from today if available
# today_actual = batch_data[batch_data["date"] == today_short].copy()
# if not today_actual.empty:
#     # Rename predicted_pm25 to pm25 for consistency in plotting function
#     today_actual = today_actual.rename(columns={'pm25': 'pm25'})
#     interpolation_df = pd.concat([today_actual[['date', 'sensor_id', 'pm25']], 
#                                  interpolation_df.rename(columns={'predicted_pm25': 'pm25'})], 
#                                 ignore_index=True)
# else:
#     interpolation_df = interpolation_df.rename(columns={'predicted_pm25': 'pm25'})

# # interpolation_df = batch_data[batch_data["date"] >= today_short]
# for i, forecast_date in enumerate(sorted(interpolation_df["date"].unique())):
#     forecast_date_short = forecast_date.strftime("%Y-%m-%d")
#     output_png = f"{interpolation_dir}/forecast_interpolation_{i}d.png"
#     print(interpolation_df.info())
    
#     plot_pm25_idw_heatmap(
#         interpolation_df,
#         locations,
#         forecast_date,
#         output_png,
#     )
#     dataset_api.upload(
#         output_png,
#         f"Resources/airquality/interpolation_{today_short}_{forecast_date_short}.png",
#         overwrite=True,
#     )

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/interpolation/forecast_interpol…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/interpolation/forecast_interpol…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/interpolation/forecast_interpol…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/interpolation/forecast_interpol…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/interpolation/forecast_interpol…

Uploading c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn/models/interpolation/forecast_interpol…